### **Documentação:** Calculadora de Quality Score (AurumQualityScoreCalculator)

#### **1. Objetivo**

Este script é um dos pilares centrais do **Pilar 1 (Qualidade Financeira)** do projeto Aurum. Sua responsabilidade é transformar os dados fundamentalistas brutos (extraídos da CVM e formatados no `fundamentals_wide.parquet`) em métricas de performance acionáveis.

O script executa um pipeline completo que:
1.  Carrega os dados brutos da CVM.
2.  Calcula os valores **TTM (Trailing Twelve Months / Últimos 12 Meses)**, corrigindo a natureza "Acumulada no Ano" (YTD) dos dados da CVM.
3.  Calcula um conjunto abrangente de **ratios financeiros** (Rentabilidade, Margens, Alavancagem, etc.) usando os dados TTM.
4.  Calcula um **Aurum Quality Score** inicial, baseado em um *ranking percentile cross-sectional* (por data) desses ratios.
5.  Salva os resultados históricos e os mais recentes.

#### **2. Configuração (Input)**

O script depende de um único arquivo de entrada, que deve ser gerado pelo pipeline de processamento da CVM:

* **`ata/cvm/final/fundamentals_wide.parquet**: Um arquivo Parquet contendo os dados de Balanço Patrimonial (BP) e Demonstração do Resultado (DRE) em formato "wide" (uma linha por empresa/data, múltiplas colunas de métricas).
    * **Importante:** O script assume que os dados da DRE (ex: `Receita Líquida`, `Lucro Bruto`) estão no formato **YTD (Acumulado no Ano)**, que é o padrão da CVM.

#### 3. Pipeline de Execução (Passo a Passo)

A classe **AurumQualityScoreCalculator** gerencia todo o fluxo de trabalho:

##### **Passo 1:** Carregar e Preparar Dados (`load_and_prepare_data`)

* Carrega o arquivo `fundamentals_wide.parquet`.
* Chama _clean_fundamentals_data para:
    * Converter DT_FIM_EXERC para datetime.
    * Remover quaisquer linhas duplicadas por CNPJ_CIA e DT_FIM_EXERC.
    * **Ordenar** os dados por CNPJ_CIA e DT_FIM_EXERC, o que é **essencial** para o cálculo de TTM.

##### **Passo 2:** Cálculo de TTM (`calculate_ttm_data`)

Este é o passo mais crítico do script. Ele converte os dados de fluxo (DRE) de YTD para TTM.

* **Lógica:** O script agrupa por `CNPJ_CIA` e aplica a função `rolling_sum_group`.
* **Desacumulação (YTD -> Trimestral):** Para cada coluna de DRE, ele calcula o valor trimestral fazendo:
    quarterly = group[col] - group[col].shift(1).fillna(0)
* **Anualização (Trimestral -> TTM):** Em seguida, ele calcula a soma móvel dos últimos 4 valores trimestrais:
    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
* O resultado é salvo em self.ratios_df, pronto para o cálculo dos ratios.

##### Passo 3: Cálculo de Ratios Financeiros (`calculate_financial_ratios`)

* Usando o self.ratios_df (que agora contém as colunas `_ttm`), este método calcula os principais indicadores financeiros.
* **Categorias:**
    * **Contas de Dívida: Dívida Bruta, Dívida Líquida, Capital Investido**.
    * **Rentabilidade (com TTM):** ROE, ROA, ROIC.
    * **Margens (com TTM):** MARGEM_EBIT, MARGEM_LIQUIDA, MARGEM_BRUTA.
    * **Alavancagem:** ALAVANCAGEM (Passivo/Ativo), DIVIDA_PL, DIVIDA_LIQ_EBIT.
    * **Liquidez:** LIQUIDEZ_CORRENTE.
    * **Eficiência (com TTM):** GIRO_ATIVO.
* Usa safe_divide para evitar erros de divisão por zero.

##### Passo 3.1: Tratamento de Outliers (`_handle_ratio_outliers`)

* Após o cálculo, os ratios são limpos.
* Valores `infinitos` são substituídos por `NaN`.
* Os dados são "winsorizados": valores extremos são "clipados" (limitados) aos **percentis 1% (inferior) e 99% (superior)**. Isso torna o scoring subsequente mais robusto.

##### Passo 4: Cálculo do Score de Qualidade (`calculate_quality_scores`)

Este método implementa a **Metodologia de Ranking Cross-Sectional**.

* **Configuração:** Define as métricas, pesos e a direção (ex: `ROIC` -> maior é melhor, `DIVIDA_LIQ_EBIT` -> menor é melhor).
* **Ranking por Data:** O script agrupa os dados por `DT_FIM_EXERC` (`grouped_by_date`).
* Para cada métrica, ele calcula o **ranking percentile** de cada empresa *dentro daquele período*:
    `scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100`
* **Score Composto:** O `aurum_quality_score` é calculado como a soma ponderada desses rankings (percentis de 0 a 100). Valores `NaN` são preenchidos com a mediana (50) para não penalizar empresas excessivamente.
* **Classificação Final:** O script calcula `quality_quintile` e `quality_grade` (A-E) com base no ranking *final* do `aurum_quality_score` de cada período.

##### Passo 5 e 6: Salvar Resultados (`get_latest_scores` e `save_results`)

* O script gera e salva quatro arquivos distintos no diretório `data/aurum_scores_output/`.

#### 6. Saída (Output)

A execução do script gera os seguintes arquivos em `data/aurum_scores_output/`:

1.  **`aurum_quality_scores_complete.parquet`**:
    * O arquivo mais importante. Contém o **histórico completo** de todas as empresas, datas, ratios calculados e scores. Este será o input para o `AurumScoringSystem` avançado e para o *backtesting*.

2.  **`aurum_quality_scores_latest.parquet`**:
    * Um arquivo de conveniência que contém apenas o **último registro (data mais recente)** de score/ratios para cada empresa.

3.  **`aurum_quality_scores_latest.csv`**:
    * Versão CSV do arquivo acima, para fácil visualização em planilhas.

4.  **`aurum_scores_statistics.txt`**:
    * Um relatório de texto (`.txt`) legível, mostrando as estatísticas do último período (distribuição de notas, Top 10 empresas) para uma verificação rápida.

---

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
tqdm.pandas()

class AurumQualityScoreCalculator:
    """ Calcula TTM, ratios financeiros (incluindo ROIC correto) e scores """

    def __init__(self, fundamentals_path: str):
        self.fundamentals_path = Path(fundamentals_path)
        self.fundamentals_df = None
        self.ratios_df = None
        self.scores_df = None
        self.dre_cols = [
            'Receita Líquida', 'Custo dos Bens e/ou Serviços Vendidos',
            'Lucro Bruto', 'EBIT', 'EBT', 'Lucro Líquido Consolidado'
        ]

    def load_and_prepare_data(self) -> pd.DataFrame:
        logger.info("📥 1. CARREGANDO NOVO fundamentals_wide.parquet...")
        try:
            self.fundamentals_df = pd.read_parquet(self.fundamentals_path)
            logger.info(f"✅ Dados carregados: {self.fundamentals_df.shape}")
            logger.info(f"Colunas encontradas: {self.fundamentals_df.columns.tolist()}")
            # Verificar se as novas colunas estão presentes
            new_cols = ['Caixa e Equivalentes', 'Dívida Curto Prazo', 'Dívida Longo Prazo']
            missing_new = [col for col in new_cols if col not in self.fundamentals_df.columns]
            if missing_new:
                 logger.error(f"❌ ERRO: Novas colunas {missing_new} NÃO encontradas no input!")
                 raise ValueError(f"Novas colunas faltando: {missing_new}")
            else:
                 logger.info("✅ Novas colunas (Caixa, Dívida CP, Dívida LP) encontradas!")

            self.fundamentals_df = self._clean_fundamentals_data(self.fundamentals_df)
            return self.fundamentals_df
        except Exception as e:
            logger.error(f"❌ Erro ao carregar dados: {e}")
            raise

    def _clean_fundamentals_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        df_clean['DT_FIM_EXERC'] = pd.to_datetime(df_clean['DT_FIM_EXERC'], errors='coerce')
        df_clean = df_clean.dropna(subset=['DT_FIM_EXERC'])
        df_clean = df_clean.drop_duplicates(subset=['CNPJ_CIA', 'DT_FIM_EXERC'])
        df_clean = df_clean.sort_values(['CNPJ_CIA', 'DT_FIM_EXERC'])
        return df_clean

    def calculate_ttm_data(self) -> pd.DataFrame:
        if self.fundamentals_df is None: raise ValueError("Dados não carregados.")
        logger.info("⏳ 2. CALCULANDO TTM...")
        df_ttm = self.fundamentals_df.copy()
        def rolling_sum_group(group):
            for col in self.dre_cols:
                if col in group.columns:
                    quarterly = group[col] - group[col].shift(1).fillna(0)
                    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
                else: group[f'{col}_ttm'] = np.nan
            return group
        grouped = df_ttm.groupby('CNPJ_CIA', group_keys=False)
        self.ratios_df = grouped.progress_apply(rolling_sum_group)
        logger.info(f"✅ TTM calculado.")
        return self.ratios_df

    def calculate_financial_ratios(self) -> pd.DataFrame:
        """ PASSO 3: Calcula ratios (com ROIC correto e ratios de dívida) """
        if self.ratios_df is None: raise ValueError("Dados TTM não calculados.")
        logger.info("🧮 3. CALCULANDO RATIOS FINANCEIROS (COM TTM)...")
        df = self.ratios_df.copy()

        def get_col(df, col_name): return df.get(col_name, np.nan)
        def safe_divide(num, den): return np.where(den == 0, np.nan, num / den)

        # --- Contas de Dívida (USANDO AS NOVAS COLUNAS) ---
        divida_cp = get_col(df, 'Dívida Curto Prazo').fillna(0) # Usando a coluna correta
        divida_lp = get_col(df, 'Dívida Longo Prazo').fillna(0) # Usando a coluna correta
        caixa = get_col(df, 'Caixa e Equivalentes').fillna(0)   # Usando a coluna correta
        pl = get_col(df, 'Patrimônio Líquido Consolidado').fillna(0)
        
        df['Dívida Bruta'] = divida_cp + divida_lp
        df['Capital Investido'] = df['Dívida Bruta'] + pl # Definição correta
        df['Dívida Líquida'] = df['Dívida Bruta'] - caixa # Definição correta
        logger.info("✅ Dívida Bruta, Capital Investido e Dívida Líquida calculados.")

        # --- RENTABILIDADE ---
        logger.info("📈 Calculando RENTABILIDADE...")
        df['ROE'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), pl)
        df['ROA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Ativo Total'))
        # <<< ROIC CORRETO >>>
        df['ROIC'] = safe_divide(get_col(df, 'EBIT_ttm'), df['Capital Investido'])
        logger.info(f"✅ ROIC (correto) calculado. Média: {df['ROIC'].mean():.4f}")
        # <<< FIM ROIC CORRETO >>>

        # --- MARGENS ---
        df['MARGEM_EBIT'] = safe_divide(get_col(df, 'EBIT_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_LIQUIDA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_BRUTA'] = safe_divide(get_col(df, 'Lucro Bruto_ttm'), get_col(df, 'Receita Líquida_ttm'))

        # --- ALAVANCAGEM ---
        logger.info("⚖️ Calculando ALAVANCAGEM...")
        df['ALAVANCAGEM'] = safe_divide(get_col(df, 'Passivo Total'), get_col(df, 'Ativo Total'))
        df['DIVIDA_PL'] = safe_divide(df['Dívida Bruta'], pl) # Agora pode ser calculado
        df['DIVIDA_LIQ_EBIT'] = safe_divide(df['Dívida Líquida'], get_col(df, 'EBIT_ttm')) # Agora pode ser calculado
        logger.info("✅ Ratios de alavancagem (incluindo DIVIDA_PL, DIVIDA_LIQ_EBIT) calculados.")

        # --- LIQUIDEZ ---
        df['LIQUIDEZ_CORRENTE'] = safe_divide(get_col(df, 'Ativo Circulante'), get_col(df, 'Passivo Circulante'))

        # --- EFICIÊNCIA ---
        df['GIRO_ATIVO'] = safe_divide(get_col(df, 'Receita Líquida_ttm'), get_col(df, 'Ativo Total'))

        self.ratios_df = self._handle_ratio_outliers(df)
        logger.info("🎯 RATIOS CALCULADOS e limpos.")
        logger.info(f"Colunas FINAIS de Ratios: {self.ratios_df.columns.tolist()}")
        return self.ratios_df

    def _handle_ratio_outliers(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        # Lista completa agora que temos os dados
        ratio_columns = [
            'ROE', 'ROA', 'ROIC', 'MARGEM_EBIT', 'MARGEM_LIQUIDA', 'MARGEM_BRUTA',
            'ALAVANCAGEM', 'DIVIDA_PL', 'DIVIDA_LIQ_EBIT', 'LIQUIDEZ_CORRENTE', 'GIRO_ATIVO'
        ]
        for col in ratio_columns:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
                if df_clean[col].notna().sum() > 0:
                    lower = df_clean[col].quantile(0.01)
                    upper = df_clean[col].quantile(0.99)
                    df_clean[col] = df_clean[col].clip(lower=lower, upper=upper)
        return df_clean

    def calculate_quality_scores(self) -> pd.DataFrame:
        """ PASSO 4: Calcula scores individuais e score composto """
        if self.ratios_df is None: raise ValueError("Ratios não calculados.")
        logger.info("🎯 4. CALCULANDO SCORES DE QUALIDADE...")
        scores_df = self.ratios_df.copy()
        grouped_by_date = scores_df.groupby('DT_FIM_EXERC')

        # Configuração original (ou ajuste como preferir)
        metrics_config = {
            'ROIC': {'direction': 1, 'weight': 0.25}, # ROIC correto
            'ROE': {'direction': 1, 'weight': 0.15},
            'MARGEM_EBIT': {'direction': 1, 'weight': 0.15},
            'MARGEM_LIQUIDA': {'direction': 1, 'weight': 0.10},
            'DIVIDA_LIQ_EBIT': {'direction': -1, 'weight': 0.15}, # Agora existe
            'LIQUIDEZ_CORRENTE': {'direction': 1, 'weight': 0.10},
            'GIRO_ATIVO': {'direction': 1, 'weight': 0.10},
            # 'ALAVANCAGEM': {'direction': -1, 'weight': 0.0}, # Pode remover ou ajustar peso
        }
        # Validar e ajustar pesos para somar 1.0
        total_w = sum(c['weight'] for c in metrics_config.values())
        if abs(total_w - 1.0) > 0.01:
             logger.warning(f"Soma dos pesos é {total_w:.2f}. Ajustando proporcionalmente...")
             for k in metrics_config: metrics_config[k]['weight'] /= total_w

        logger.info(f"Configuração de métricas para score: { {k: v['weight'] for k, v in metrics_config.items()} }")

        for metric, config in metrics_config.items():
            if metric in scores_df.columns and scores_df[metric].notna().any():
                score_col = f'score_{metric}'
                if config['direction'] == 1:
                    scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100
                else:
                    scores_df[score_col] = grouped_by_date[metric].rank(ascending=False, pct=True) * 100
                logger.info(f"  ✅ Score {metric} calculado.")
            else:
                 logger.warning(f"Métrica '{metric}' não encontrada ou sem dados para score.")

        logger.info("⚖️ Calculando score composto...")
        scores_df['aurum_quality_score'] = 0.0
        total_applied_weight = 0.0
        for metric, config in metrics_config.items():
            score_col = f'score_{metric}'
            if score_col in scores_df.columns:
                scores_df['aurum_quality_score'] += scores_df[score_col].fillna(50) * config['weight']
                total_applied_weight += config['weight']

        if total_applied_weight > 0:
            scores_df['aurum_quality_score'] /= total_applied_weight

        # ... (código de classificação e restante igual ao anterior) ...
        logger.info("🏆 Classificando empresas...")
        try:
             valid_scores = scores_df['aurum_quality_score'].dropna()
             if not valid_scores.empty:
                  ranks = valid_scores.rank(ascending=False, pct=True)
                  scores_df['final_rank'] = ranks
                  try:
                       scores_df['quality_quintile'] = pd.qcut(scores_df['final_rank'].dropna(), 5, labels=[f'{i}º Quintil' for i in range(1, 6)])
                  except ValueError: scores_df['quality_quintile'] = pd.cut(scores_df['final_rank'].dropna(), 5, labels=False)
                  scores_df['quality_grade'] = pd.cut(scores_df['final_rank'].dropna(), bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=['A', 'B', 'C', 'D', 'E'], right=True, include_lowest=True)
             else: scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        except Exception as e:
             logger.error(f"Erro ao calcular ranks/grades: {e}")
             scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        
        self.scores_df = scores_df
        logger.info(f"Colunas FINAIS ANTES de salvar: {self.scores_df.columns.tolist()}")
        return scores_df

    # Métodos get_latest_scores, save_results, _save_statistics são iguais aos anteriores
    # ... (Copie e cole os métodos get_latest_scores, save_results, _save_statistics da versão anterior aqui) ...
    def get_latest_scores(self) -> pd.DataFrame:
        if self.scores_df is None: raise ValueError("Scores não calculados.")
        latest_scores = self.scores_df.sort_values('DT_FIM_EXERC').groupby('CNPJ_CIA').last().reset_index()
        return latest_scores

    def save_results(self, output_dir: str = "data/aurum_scores"):
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        if self.scores_df is not None:
            logger.info(f"Salvando DataFrame com colunas: {self.scores_df.columns.tolist()}")
            if 'ROIC' not in self.scores_df.columns: logger.error("ERRO FATAL: Coluna 'ROIC' AUSENTE antes de salvar!")
            scores_path = output_path / "aurum_quality_scores_complete.parquet"
            self.scores_df.to_parquet(scores_path, index=False)
            logger.info(f"💾 Scores completos (histórico) salvos: {scores_path}")
        latest_scores = self.get_latest_scores()
        latest_path = output_path / "aurum_quality_scores_latest.parquet"
        latest_csv_path = output_path / "aurum_quality_scores_latest.csv"
        latest_scores.to_parquet(latest_path, index=False)
        latest_scores.to_csv(latest_csv_path, index=False, sep=';', encoding='utf-8-sig')
        logger.info(f"💾 Scores mais recentes salvos: {latest_path} e {latest_csv_path}")
        stats_path = output_path / "aurum_scores_statistics.txt"
        self._save_statistics(latest_scores, stats_path)
        logger.info(f"💾 Estatísticas salvas: {stats_path}")
        return {'complete_scores': scores_path,'latest_scores': latest_path,'statistics': stats_path}

    def _save_statistics(self, scores_df: pd.DataFrame, stats_path: Path):
        with open(stats_path, 'w', encoding='utf-8') as f:
            f.write("AURUM QUALITY SCORE - ESTATÍSTICAS (ÚLTIMO PERÍODO)\n" + "=" * 50 + "\n\n")
            f.write(f"Total de empresas: {len(scores_df)}\n")
            max_date = scores_df['DT_FIM_EXERC'].max()
            f.write(f"Período mais recente: {max_date if pd.notna(max_date) else 'N/A'}\n\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO DOS SCORES:\n" + f"  Média: {scores_df['aurum_quality_score'].mean():.2f}\n" +
                         f"  Mediana: {scores_df['aurum_quality_score'].median():.2f}\n" + f"  Mínimo: {scores_df['aurum_quality_score'].min():.2f}\n" +
                         f"  Máximo: {scores_df['aurum_quality_score'].max():.2f}\n\n")
            else: f.write("DISTRIBUIÇÃO DOS SCORES: N/A\n\n")
            if 'quality_grade' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO POR NOTAS:\n")
                 grade_counts = scores_df['quality_grade'].value_counts().sort_index(ascending=True)
                 for grade, count in grade_counts.items(): f.write(f"  Nota {grade}: {count} empresas\n")
            else: f.write("DISTRIBUIÇÃO POR NOTAS: N/A\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("\nTOP 10 EMPRESAS:\n")
                 top_10 = scores_df.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
                 for i, (_, row) in enumerate(top_10.iterrows(), 1):
                      grade = row.get('quality_grade', 'N/A')
                      f.write(f"  {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})\n")
            else: f.write("\nTOP 10 EMPRESAS: N/A\n")


# ==================== EXECUÇÃO PRINCIPAL ====================
def main():
    logger.info("🚀 INICIANDO PIPELINE DO AURUM QUALITY SCORE (Versão Final com ROIC Correto)")
    try:
        calculator = AurumQualityScoreCalculator(
            fundamentals_path="../data/cvm/final/fundamentals_wide.parquet" # Ler o NOVO input
        )
        calculator.load_and_prepare_data()
        calculator.calculate_ttm_data()
        calculator.calculate_financial_ratios() # Calcular ROIC correto e ratios de dívida
        calculator.calculate_quality_scores()   # Usar ROIC correto e ratios de dívida
        output_files = calculator.save_results(output_dir="../data/aurum_scores") # Salvar no diretório correto
        logger.info(f"💾 Resultados (com ROIC correto) salvos em: {output_files}")

        latest_scores = calculator.get_latest_scores()
        # ... (impressão dos resultados igual) ...
        print("\n" + "="*60 + "\n🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)\n" + "="*60)
        print(f"\n📊 TOTAL DE EMPRESAS: {len(latest_scores)}")
        if 'aurum_quality_score' in latest_scores.columns: print(f"📈 SCORE MÉDIO: {latest_scores['aurum_quality_score'].mean():.2f}")
        print(f"\n📋 DISTRIBUIÇÃO DAS NOTAS:")
        if 'quality_grade' in latest_scores.columns:
             grade_dist = latest_scores['quality_grade'].value_counts().sort_index(ascending=True)
             for grade, count in grade_dist.items(): print(f"   Nota {grade}: {count:3d} empresas")
        print(f"\n🥇 TOP 10 EMPRESAS:")
        if 'aurum_quality_score' in latest_scores.columns:
             top_10 = latest_scores.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
             for i, (_, row) in enumerate(top_10.iterrows(), 1):
                  grade = row.get('quality_grade', 'N/A')
                  print(f"   {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})")

        return calculator

    except Exception as e:
        logger.error(f"❌ ERRO NO PIPELINE: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

if __name__ == "__main__":
    aurum_calculator = main()

2025-12-11 01:03:08,864 - INFO - 🚀 INICIANDO PIPELINE DO AURUM QUALITY SCORE (Versão Final com ROIC Correto)
2025-12-11 01:03:08,873 - INFO - 📥 1. CARREGANDO NOVO fundamentals_wide.parquet...
2025-12-11 01:03:09,108 - INFO - ✅ Dados carregados: (21559, 19)
2025-12-11 01:03:09,113 - INFO - Colunas encontradas: ['CNPJ_CIA', 'DENOM_CIA', 'DT_FIM_EXERC', 'Custo dos Bens e/ou Serviços Vendidos', 'EBIT', 'EBT', 'Lucro Bruto', 'Lucro Líquido Consolidado', 'Receita Líquida', 'Ativo Circulante', 'Ativo Não Circulante', 'Ativo Total', 'Caixa e Equivalentes', 'Dívida Curto Prazo', 'Dívida Longo Prazo', 'Passivo Circulante', 'Passivo Não Circulante', 'Passivo Total', 'Patrimônio Líquido Consolidado']
2025-12-11 01:03:09,131 - INFO - ✅ Novas colunas (Caixa, Dívida CP, Dívida LP) encontradas!
2025-12-11 01:03:09,274 - INFO - ⏳ 2. CALCULANDO TTM...
100%|██████████| 729/729 [00:05<00:00, 123.56it/s]
2025-12-11 01:03:15,197 - INFO - ✅ TTM calculado.
2025-12-11 01:03:15,200 - INFO - 🧮 3. CALCULANDO RATI


🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)

📊 TOTAL DE EMPRESAS: 729
📈 SCORE MÉDIO: 49.27

📋 DISTRIBUIÇÃO DAS NOTAS:
   Nota A: 191 empresas
   Nota B:  90 empresas
   Nota C:  83 empresas
   Nota D: 147 empresas
   Nota E: 218 empresas

🥇 TOP 10 EMPRESAS:
    1. CAMIL ALIMENTOS S.A.                100.00 (Nota A)
    2. MINUPAR PARTICIPACOES S.A.           90.07 (Nota A)
    3. 521 PARTICIPACOES S.A. - EM LIQUIDA  89.17 (Nota A)
    4. STEIN SP II PARTICIPAÇÕES S.A.       87.37 (Nota A)
    5. COMERCIAL QUINTELLA COM EXP SA EM L  84.67 (Nota A)
    6. SONDOTECNICA ENGENHARIA SOLOS S.A.   83.36 (Nota A)
    7. EPR INFRAESTRUTURA PR S.A.           83.19 (Nota A)
    8. MÉLIUZ S.A.                          82.29 (Nota A)
    9. RIVA INCORPORADORA S.A               79.77 (Nota A)
   10. AURA ALMAS MINERAÇÃO S.A.            79.68 (Nota A)


In [ ]:
import pandas as pd
import numpy as np
import os
import logging
from pathlib import Path

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Arquivos de Input (VERIFICADOS) ---
PATH_PRECOS_LIMPOS = "data/historical/all_histories_cleaned.parquet"
PATH_PRECOS_WIDE = "data/historical/prices_close_wide.parquet"
PATH_FUNDAMENTOS = "data/aurum_scores/aurum_quality_scores_complete.parquet" # Seu arquivo de fundamentos com ROIC
PATH_SENTIMENTO = "data/news/news_with_sentiment.parquet"
PATH_DE_PARA = "data/ticker_cnpj_map.parquet" # Seu arquivo de mapeamento

# --- Arquivo de Output ---
OUTPUT_DIR = "data"
OUTPUT_FILENAME = "aurum_master_features.parquet"
output_path = Path(OUTPUT_DIR) / OUTPUT_FILENAME

def calcular_volatilidade_mensal():
    # ... (código igual, sem alterações) ...
    logger.info("Iniciando Unificação (Passo 1/3): Cálculo da Volatilidade...")
    try: df_prices_daily = pd.read_parquet(PATH_PRECOS_LIMPOS)
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_PRECOS_LIMPOS}"); return None
    df_prices_daily['date'] = pd.to_datetime(df_prices_daily['date'])
    df_prices_daily['returns'] = df_prices_daily.groupby('ticker')['Adj Close'].pct_change()
    df_prices_daily['VOLATILIDADE'] = df_prices_daily.groupby('ticker')['returns'].rolling(window=63, min_periods=30).std().reset_index(0, drop=True)
    df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()
    logger.info(f"✅ Volatilidade mensal calculada.")
    return df_vol_mensal

def agregar_sentimento_mensal():
    # ... (código igual, sem alterações) ...
    logger.info("Iniciando Unificação (Passo 2/3): Agregação de Sentimento...")
    try: df_sent_raw = pd.read_parquet(PATH_SENTIMENTO)
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_SENTIMENTO}"); return None
    df_sent_raw = df_sent_raw.rename(columns={'ticker_query': 'ticker', 'published_date': 'date'})
    df_sent_raw['date'] = pd.to_datetime(df_sent_raw['date'], utc=True).dt.tz_localize(None)
    if not df_sent_raw['ticker'].str.contains('.SA').any():
        logger.warning("Tickers sem sufixo .SA. Adicionando...")
        df_sent_raw['ticker'] = df_sent_raw['ticker'].apply(lambda x: f"{x}.SA" if not str(x).endswith(".SA") else x)
    df_sent_mensal = df_sent_raw.set_index('date').groupby('ticker').resample('M').agg(
        SENTIMENT_MEDIO=('numeric_sentiment', 'mean'),
        SENTIMENT_STD=('numeric_sentiment', 'std'),
        NEWS_COUNT=('ticker', 'count')
    ).reset_index()
    logger.info(f"✅ Sentimento mensal agregado.")
    return df_sent_mensal

def unificar_dataframe_mestre(df_vol_mensal, df_sent_mensal):
    if df_vol_mensal is None or df_sent_mensal is None: return
    logger.info("Iniciando Unificação (Passo 3/3): Junção do DataFrame Mestre...")

    # --- 1. Carregar Base de Preços Mensal ---
    try:
        df_close_wide = pd.read_parquet(PATH_PRECOS_WIDE)
        df_base_mensal = df_close_wide.melt(ignore_index=False, var_name='ticker', value_name='Adj Close').reset_index()
        df_base_mensal = df_base_mensal.rename(columns={'index': 'date'})
        logger.info(f"Base de preços (wide) carregada.")
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_PRECOS_WIDE}"); return

    # --- 2. Carregar Fundamentos (Trimestrais) ---
    try:
        df_fund = pd.read_parquet(PATH_FUNDAMENTOS)
        df_fund = df_fund.rename(columns={'DT_FIM_EXERC': 'date'})
        df_fund['date'] = pd.to_datetime(df_fund['date'])
        logger.info(f"Fundamentos carregados. Colunas: {df_fund.columns.tolist()}")
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_FUNDAMENTOS}"); return
    except KeyError as e: logger.error(f"Erro ao renomear 'DT_FIM_EXERC': {e}"); return

    # --- 3. Carregar o Mapeamento (DE-PARA) ---
    try:
        df_mapping = pd.read_parquet(PATH_DE_PARA)
        if 'ticker' not in df_mapping.columns or 'CNPJ_CIA' not in df_mapping.columns:
             logger.error(f"ERRO: Mapeamento {PATH_DE_PARA} sem 'ticker' ou 'CNPJ_CIA'.")
             return
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_DE_PARA}"); return

    df_fund_com_ticker = pd.merge(df_fund, df_mapping, on='CNPJ_CIA', how='left')
    df_fund_com_ticker = df_fund_com_ticker.dropna(subset=['ticker'])
    logger.info("Fundamentos mapeados para tickers.")

    # --- 4. Construir o DataFrame Mestre ---
    df_master = df_base_mensal.sort_values(by='date')
    df_fund_com_ticker = df_fund_com_ticker.sort_values(by='date')

    # 4a. Juntar Fundamentos (merge_asof)
    df_master = pd.merge_asof(
        df_master, df_fund_com_ticker, on='date', by='ticker', direction='backward'
    )
    logger.info("Merge 'as-of' dos fundamentos concluído.")
    logger.info(f"Colunas no df_master APÓS merge_asof: {df_master.columns.tolist()}") # DEBUG

    # 4b. Juntar Volatilidade (merge)
    df_master = pd.merge(df_master, df_vol_mensal, on=['date', 'ticker'], how='left')
    logger.info("Merge da volatilidade concluído.")

    # 4c. Juntar Sentimento (merge)
    df_master = pd.merge(df_master, df_sent_mensal, on=['date', 'ticker'], how='left')
    logger.info("Merge do sentimento concluído.")

    # --- 5. Limpeza Final ---
    df_master['SENTIMENT_MEDIO'] = df_master['SENTIMENT_MEDIO'].fillna(0)
    df_master['VOLATILIDADE'] = df_master.groupby('ticker')['VOLATILIDADE'].ffill().bfill()

    # <<< VERSÃO FINAL DO dropna >>>
    # Usando ambas as colunas, pois df_master.info() provou que elas existem
    fundamental_key_columns = ['ROE', 'ROIC'] # <--- USANDO AMBAS AS COLUNAS

    missing_cols = [col for col in fundamental_key_columns if col not in df_master.columns]
    if missing_cols:
        # Este erro não deve mais acontecer baseado no seu df.info()
        logger.error(f"ERRO CRÍTICO PÓS-MERGE: Colunas {missing_cols} não encontradas!")
        return
    else:
        logger.info(f"Aplicando dropna nas colunas chave: {fundamental_key_columns}")
        df_master = df_master.dropna(subset=fundamental_key_columns)
    # <<< FIM DA VERSÃO FINAL >>>

    logger.info(f"Limpeza final concluída. DataFrame Mestre pronto com {len(df_master)} linhas.")

    # --- 6. Salvar o Novo DataFrame Mestre ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df_master.to_parquet(output_path, index=False)
    logger.info(f"✅✅✅ DataFrame Mestre salvo em: {output_path} ✅✅✅")

    print("\n--- Informações do DataFrame Mestre Gerado ---")
    df_master.info(verbose=True, show_counts=True) # Mostrar detalhes
    print("\n--- Amostra do DataFrame Mestre ---")
    sample_cols = ['date', 'ticker', 'ROE', 'ROIC', 'SENTIMENT_MEDIO', 'VOLATILIDADE']
    print(df_master.sample(5)[[col for col in sample_cols if col in df_master.columns]])

    return df_master

# --- Execução Principal ---
if __name__ == "__main__":
    df_vol = calcular_volatilidade_mensal()
    df_sent = agregar_sentimento_mensal()
    if df_vol is not None and df_sent is not None:
        unificar_dataframe_mestre(df_vol, df_sent)
    else:
        logger.error("Falha ao gerar dados. O DataFrame Mestre não foi criado.")

### **Documentação:** Sistema de Scoring Avançado (AurumScoringSystem)

#### 1. Objetivo

Este script é o "cérebro" do projeto Aurum, onde a tese de investimento quantitativo é de fato implementada. Ele representa a **evolução** do `AurumQualityScoreCalculator`, aplicando uma metodologia de scoring mais robusta e academicamente embasada.

Enquanto o script anterior (`Calculator`) era focado em *calcular ratios* e criar um *score de ranking simples*, este script (`System`) tem como responsabilidades:
1.  **Carregar uma Tese:** Define um conjunto explícito de **métricas e pesos** baseados em teorias financeiras (ex: Fama & French, Graham & Dodd), armazenados na classe `ScoringMetric`.
2.  **Normalização Avançada:** Substitui a normalização por ranking (percentil) por uma **normalização sigmóide (baseada em Z-Score)**. Isso cria um score mais suave, robusto a outliers e que recompensa melhor empresas excepcionais.
3.  **Incorporar Fatores Múltiplos:** O sistema é projetado para consumir *não apenas* os ratios financeiros do script anterior, mas também métricas de **crescimento**, **sentimento (NLP)** e **volatilidade (preço)**.
4.  **Validar-se:** Gera um relatório de validação que compara a contribuição *teórica* de cada métrica (o peso que definimos) com sua contribuição *real* no score final.
5.  **Gerar o Score Final:** Calcula o `aurum_quality_score` final e as classificações (A, B, C, D, E).

#### 2. Configuração (Input)

Este script é o **segundo passo** no pipeline de scoring e depende da saída do script anterior.

1.  **Input Principal (Obrigatório):**
    * `data/aurum_scores/aurum_quality_scores_complete.parquet`: Este é o arquivo de **output** gerado pelo `AurumQualityScoreCalculator`. Ele contém todos os ratios históricos (ROE, ROIC, etc.) já calculados e tratados.

2.  **Input Opcional (Configuração):**
    * O construtor `AurumScoringSystem(config_path="...")` aceita um caminho para um arquivo `.json`. Isso permite carregar um conjunto personalizado de métricas e pesos sem alterar o código, facilitando a experimentação. Se nenhum caminho for fornecido, ele usa os pesos padrão definidos em `_initialize_scoring_metrics`.

#### 3. Saída (Output)

O script cria um novo diretório: `data/aurum_final_scores/`.

1.  **`aurum_advanced_scores.parquet`**:
    * O **histórico completo** de todas as empresas com os scores finais (0-100), notas (A-E) e quintis. Este é o arquivo final que será usado para o **backtesting**.

2.  **`aurum_latest_advanced_scores.parquet`**:
    * Um snapshot contendo apenas o **último score disponível** para cada empresa.

3.  **`aurum_scoring_config.json`**:
    * Um arquivo `.json` que salva a tese exata (métricas e pesos) usada nesta execução, garantindo a reprodutibilidade dos resultados.

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional, Tuple
import warnings
from dataclasses import dataclass
import json

warnings.filterwarnings('ignore')

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class ScoringMetric:
    """Classe para representar uma métrica de scoring"""
    name: str
    weight: float
    direction: int  # 1 = maior é melhor, -1 = menor é melhor
    description: str
    min_value: float = None
    max_value: float = None
    ideal_range: Tuple[float, float] = None

class AurumScoringSystem:
    """
    Sistema avançado de scoring para o Aurum Quality Score
    com pesos baseados em fundamentos financeiros
    """
    
    def __init__(self, config_path: str = None):
        self.scoring_metrics = self._initialize_scoring_metrics(config_path)
        self.quality_thresholds = {
            'A': 80,  # Excelente
            'B': 60,  # Bom
            'C': 40,  # Regular
            'D': 20,  # Ruim
            'E': 0    # Muito ruim
        }
        
    def _initialize_scoring_metrics(self, config_path: str = None) -> Dict[str, ScoringMetric]:
        """
        Inicializa as métricas de scoring com pesos baseados em:
        - Graham & Dodd: Security Analysis
        - Fama & French: Three Factor Model  
        - Práticas do mercado quantitativo
        """
        
        if config_path and Path(config_path).exists():
            return self._load_custom_config(config_path)
        
        # Pesos baseados em importância relativa para qualidade de empresas
        # CORREÇÃO: Soma total = 1.0 (100%)
        metrics_config = {
            # === RENTABILIDADE (47% do total) ===
            'ROE': ScoringMetric(
                name='ROE', weight=0.18, direction=1,  # Aumentado de 0.15 para 0.18
                description='Return on Equity - Eficiência do capital próprio',
                ideal_range=(0.10, 0.25)
            ),
            'ROA': ScoringMetric(
                name='ROA', weight=0.12, direction=1,
                description='Return on Assets - Eficiência dos ativos',
                ideal_range=(0.05, 0.15)
            ),
            'MARGEM_EBIT': ScoringMetric(
                name='MARGEM_EBIT', weight=0.08, direction=1,
                description='Margem Operacional - Ebit/Receita',
                ideal_range=(0.08, 0.20)
            ),
            'MARGEM_LIQUIDA': ScoringMetric(
                name='MARGEM_LIQUIDA', weight=0.05, direction=1,  # Reduzido de 0.06 para 0.05
                description='Margem Líquida - Lucro/Receita',
                ideal_range=(0.06, 0.18)
            ),
            'MARGEM_BRUTA': ScoringMetric(
                name='MARGEM_BRUTA', weight=0.04, direction=1,
                description='Margem Bruta - Lucro Bruto/Receita',
                ideal_range=(0.20, 0.50)
            ),
            
            # === SOLVÊNCIA E ALAVANCAGEM (33% do total) ===
            'ALAVANCAGEM': ScoringMetric(
                name='ALAVANCAGEM', weight=0.09, direction=-1,  # Aumentado de 0.08 para 0.09
                description='Alavancagem Total - Passivo/Ativo',
                ideal_range=(0.30, 0.60)
            ),
            'DIVIDA_PL': ScoringMetric(
                name='DIVIDA_PL', weight=0.09, direction=-1,  # Aumentado de 0.08 para 0.09
                description='Dívida/Patrimônio Líquido',
                ideal_range=(0.50, 1.50)
            ),
            'LIQUIDEZ_CORRENTE': ScoringMetric(
                name='LIQUIDEZ_CORRENTE', weight=0.08, direction=1,  # Aumentado de 0.07 para 0.08
                description='Liquidez Corrente - Ativo Circulante/Passivo Circulante',
                ideal_range=(1.20, 3.00)
            ),
            'GIRO_ATIVO': ScoringMetric(
                name='GIRO_ATIVO', weight=0.07, direction=1,
                description='Giro do Ativo - Receita/Ativo Total',
                ideal_range=(0.30, 1.00)
            ),
            
            # === CRESCIMENTO (10% do total) ===
            'CRESC_RECEITA': ScoringMetric(
                name='CRESC_RECEITA', weight=0.05, direction=1,
                description='Crescimento da Receita (anual)',
                ideal_range=(0.05, 0.30)
            ),
            'CRESC_LUCRO': ScoringMetric(
                name='CRESC_LUCRO', weight=0.05, direction=1,
                description='Crescimento do Lucro Líquido (anual)',
                ideal_range=(0.08, 0.40)
            ),
            
            # === EFICIÊNCIA (10% do total) ===
            'SENTIMENT_MEDIO': ScoringMetric(
                name='SENTIMENT_MEDIO', weight=0.05, direction=1,
                description='Sentimento Médio de Notícias',
                ideal_range=(0.10, 0.80)
            ),
            'VOLATILIDADE': ScoringMetric(
                name='VOLATILIDADE', weight=0.05, direction=-1,
                description='Volatilidade dos Retornos (3 meses)',
                ideal_range=(0.10, 0.40)
            )
        }
        
        # Validar que soma dos pesos = 1.0
        total_weight = sum(metric.weight for metric in metrics_config.values())
        if abs(total_weight - 1.0) > 0.001:
            # Ajuste automático para garantir soma = 1.0
            adjustment_factor = 1.0 / total_weight
            for metric in metrics_config.values():
                metric.weight *= adjustment_factor
            
            logger.info(f"🔧 Pesos ajustados automaticamente para soma = 1.0")
        
        logger.info(f"✅ Sistema de scoring inicializado com {len(metrics_config)} métricas")
        logger.info(f"📊 Soma dos pesos: {sum(metric.weight for metric in metrics_config.values()):.3f}")
        
        return metrics_config
    
    def _load_custom_config(self, config_path: str) -> Dict[str, ScoringMetric]:
        """Carrega configuração personalizada de pesos"""
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                config_data = json.load(f)
            
            metrics_config = {}
            for metric_name, metric_data in config_data.items():
                metrics_config[metric_name] = ScoringMetric(**metric_data)
            
            logger.info(f"✅ Configuração personalizada carregada: {config_path}")
            return metrics_config
            
        except Exception as e:
            logger.error(f"❌ Erro ao carregar configuração: {e}")
            return self._initialize_scoring_metrics()  # Fallback para padrão
    
    def calculate_individual_scores(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula scores individuais para cada métrica usando normalização avançada
        """
        scores_df = df.copy()
        
        logger.info("🎯 Calculando scores individuais...")
        
        for metric_name, metric_config in self.scoring_metrics.items():
            if metric_name not in scores_df.columns:
                logger.warning(f"⚠️ Métrica {metric_name} não encontrada no dataset")
                continue
            
            # CORREÇÃO: Verificação de tipo de dados corrigida para NumPy 2.0
            if scores_df[metric_name].dtype == object or pd.api.types.is_string_dtype(scores_df[metric_name]):
                logger.warning(f"⚠️ Métrica {metric_name} é do tipo texto - pulando")
                continue
            
            # Remover outliers extremos
            clean_series = self._remove_outliers(scores_df[metric_name])
            
            if metric_config.direction == 1:
                # MAIOR é melhor - usar função sigmoide para suavizar
                scores_df[f'score_{metric_name}'] = self._sigmoid_normalization(clean_series)
            else:
                # MENOR é melhor - inverter a normalização
                scores_df[f'score_{metric_name}'] = 1 - self._sigmoid_normalization(clean_series)
            
            # Aplicar pesos
            scores_df[f'score_{metric_name}'] *= 100  # Converter para 0-100
            scores_df[f'score_{metric_name}'] *= metric_config.weight
            
            valid_scores = scores_df[f'score_{metric_name}'].notna().sum()
            logger.info(f"  ✅ {metric_name}: {valid_scores} scores calculados (peso: {metric_config.weight*100:.1f}%)")
        
        return scores_df
    
    def _remove_outliers(self, series: pd.Series, n_std: int = 3) -> pd.Series:
        """Remove outliers usando método Z-score"""
        # CORREÇÃO: Verificação de tipo de dados atualizada para NumPy 2.0
        if series.dtype == object or pd.api.types.is_string_dtype(series) or pd.api.types.is_categorical_dtype(series):
            return series
        
        try:
            z_scores = np.abs((series - series.mean()) / series.std())
            clean_series = series.copy()
            clean_series[z_scores > n_std] = np.nan
            
            outliers_removed = (z_scores > n_std).sum()
            if outliers_removed > 0:
                logger.debug(f"  🎯 {outliers_removed} outliers removidos de {series.name}")
            
            return clean_series
        except Exception as e:
            logger.warning(f"  ⚠️ Erro ao remover outliers de {series.name}: {e}")
            return series
    
    def _sigmoid_normalization(self, series: pd.Series) -> pd.Series:
        """
        Normalização usando função sigmoide para suavizar valores extremos
        Mais robusta que rankeamento simples
        """
        # CORREÇÃO: Verificação de tipo de dados atualizada
        if series.dtype == object or pd.api.types.is_string_dtype(series) or pd.api.types.is_categorical_dtype(series):
            return series
        
        try:
            # Standardizar para média 0, std 1
            standardized = (series - series.mean()) / series.std()
            
            # Aplicar sigmoide
            sigmoid = 1 / (1 + np.exp(-standardized))
            
            return sigmoid
        except Exception as e:
            logger.warning(f"  ⚠️ Erro na normalização sigmoide de {series.name}: {e}")
            # Fallback: normalização linear simples
            return (series - series.min()) / (series.max() - series.min())
    
    def calculate_composite_score(self, scores_df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula o score composto final com validações
        """
        logger.info("⚖️ Calculando score composto...")
        
        # Identificar colunas de score
        score_columns = [col for col in scores_df.columns if col.startswith('score_')]
        
        if not score_columns:
            raise ValueError("❌ Nenhuma coluna de score encontrada")
        
        logger.info(f"📋 Colunas de score encontradas: {len(score_columns)}")
        
        # Calcular score composto
        scores_df['aurum_quality_score'] = scores_df[score_columns].sum(axis=1, skipna=True)
        
        # Normalizar para 0-100 (caso alguns pesos não tenham sido aplicados)
        max_possible_score = sum(metric.weight * 100 for metric in self.scoring_metrics.values())
        scores_df['aurum_quality_score'] = (scores_df['aurum_quality_score'] / max_possible_score) * 100
        
        # Garantir que scores estejam entre 0 e 100
        scores_df['aurum_quality_score'] = scores_df['aurum_quality_score'].clip(0, 100)
        
        # Aplicar classificações
        scores_df = self._apply_quality_classifications(scores_df)
        
        valid_scores = scores_df['aurum_quality_score'].notna().sum()
        logger.info(f"✅ Score composto calculado: {valid_scores} empresas")
        
        return scores_df
    
    def _apply_quality_classifications(self, df: pd.DataFrame) -> pd.DataFrame:
        """Aplica classificações de qualidade (A, B, C, D, E)"""
        
        # Classificação por quintis
        try:
            df['quality_quintile'] = pd.qcut(
                df['aurum_quality_score'], 
                5, 
                labels=['5º Quintil', '4º Quintil', '3º Quintil', '2º Quintil', '1º Quintil']
            )
        except ValueError as e:
            logger.warning(f"⚠️ Erro no qcut, usando cortes uniformes: {e}")
            # Fallback para cortes uniformes
            df['quality_quintile'] = pd.cut(
                df['aurum_quality_score'],
                bins=5,
                labels=['5º Quintil', '4º Quintil', '3º Quintil', '2º Quintil', '1º Quintil']
            )
        
        # Classificação por letras baseada em thresholds
        conditions = [
            df['aurum_quality_score'] >= self.quality_thresholds['A'],
            df['aurum_quality_score'] >= self.quality_thresholds['B'],
            df['aurum_quality_score'] >= self.quality_thresholds['C'],
            df['aurum_quality_score'] >= self.quality_thresholds['D'],
            df['aurum_quality_score'] >= self.quality_thresholds['E']
        ]
        
        choices = ['A', 'B', 'C', 'D', 'E']
        
        df['quality_grade'] = np.select(conditions, choices, default='E')
        
        # Classificação descritiva
        grade_descriptions = {
            'A': 'Excelente Qualidade',
            'B': 'Boa Qualidade', 
            'C': 'Qualidade Regular',
            'D': 'Qualidade Baixa',
            'E': 'Qualidade Muito Baixa'
        }
        
        df['quality_description'] = df['quality_grade'].map(grade_descriptions)
        
        return df
    
    def calculate_sector_adjusted_scores(self, df: pd.DataFrame, sector_column: str = 'setor') -> pd.DataFrame:
        """
        Calcula scores ajustados por setor (quando informação de setor disponível)
        """
        if sector_column not in df.columns:
            logger.warning("⚠️ Coluna de setor não encontrada - pulando ajuste setorial")
            return df
        
        logger.info("🏭 Aplicando ajuste setorial...")
        
        df_sector_adjusted = df.copy()
        
        # Calcular medianas por setor para cada métrica
        for metric_name in self.scoring_metrics.keys():
            if metric_name in df.columns:
                sector_medians = df.groupby(sector_column)[metric_name].median()
                
                # Ajustar scores baseado na mediana do setor
                df_sector_adjusted[f'{metric_name}_sector_adj'] = df.apply(
                    lambda row: row[metric_name] / sector_medians.get(row[sector_column], 1.0), 
                    axis=1
                )
        
        # Recalcular scores com ajuste setorial
        score_columns_adj = [col for col in df_sector_adjusted.columns if col.endswith('_sector_adj')]
        
        if score_columns_adj:
            logger.info(f"✅ Ajuste setorial aplicado para {len(score_columns_adj)} métricas")
        
        return df_sector_adjusted
    
    def validate_scoring_system(self, scores_df: pd.DataFrame) -> Dict:
        """
        Valida o sistema de scoring através de análises estatísticas
        """
        logger.info("🔍 Validando sistema de scoring...")
        
        validation_report = {
            'basic_stats': {
                'total_companies': len(scores_df),
                'companies_with_scores': scores_df['aurum_quality_score'].notna().sum(),
                'score_mean': scores_df['aurum_quality_score'].mean(),
                'score_std': scores_df['aurum_quality_score'].std(),
                'score_min': scores_df['aurum_quality_score'].min(),
                'score_max': scores_df['aurum_quality_score'].max(),
                'score_median': scores_df['aurum_quality_score'].median()
            },
            'distribution': {
                'grade_A': len(scores_df[scores_df['quality_grade'] == 'A']),
                'grade_B': len(scores_df[scores_df['quality_grade'] == 'B']),
                'grade_C': len(scores_df[scores_df['quality_grade'] == 'C']),
                'grade_D': len(scores_df[scores_df['quality_grade'] == 'D']),
                'grade_E': len(scores_df[scores_df['quality_grade'] == 'E'])
            },
            'correlation_analysis': {},
            'metric_contribution': {},
            'weight_summary': {}
        }
        
        # Análise de correlação entre scores individuais e score final
        score_columns = [col for col in scores_df.columns if col.startswith('score_')]
        
        for score_col in score_columns:
            if score_col in scores_df.columns:
                correlation = scores_df[score_col].corr(scores_df['aurum_quality_score'])
                validation_report['correlation_analysis'][score_col] = round(correlation, 4) if not pd.isna(correlation) else None
        
        # Contribuição de cada métrica
        total_actual_weight = 0
        for metric_name, metric_config in self.scoring_metrics.items():
            score_col = f'score_{metric_name}'
            if score_col in scores_df.columns:
                actual_contribution = scores_df[score_col].mean() / scores_df['aurum_quality_score'].mean() * 100
                if not pd.isna(actual_contribution):
                    validation_report['metric_contribution'][metric_name] = {
                        'weight': round(metric_config.weight * 100, 2),
                        'actual_contribution': round(actual_contribution, 2),
                        'description': metric_config.description
                    }
                    total_actual_weight += actual_contribution
        
        validation_report['weight_summary']['total_theoretical_weight'] = 100.0
        validation_report['weight_summary']['total_actual_weight'] = round(total_actual_weight, 2)
        
        logger.info("✅ Sistema de scoring validado")
        return validation_report
    
    def save_scoring_configuration(self, output_path: str = "data/aurum_scoring_config.json"):
        """Salva a configuração do sistema de scoring"""
        config_data = {}
        
        for metric_name, metric_config in self.scoring_metrics.items():
            config_data[metric_name] = {
                'name': metric_config.name,
                'weight': metric_config.weight,
                'direction': metric_config.direction,
                'description': metric_config.description,
                'min_value': metric_config.min_value,
                'max_value': metric_config.max_value,
                'ideal_range': metric_config.ideal_range
            }
        
        output_dir = Path(output_path).parent
        output_dir.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=2, ensure_ascii=False)
        
        logger.info(f"💾 Configuração salva em: {output_path}")
        return output_path

# ==================== EXECUÇÃO PRINCIPAL ====================

def run_advanced_scoring_system():
    """
    Executa o sistema avançado de scoring completo
    """
    logger.info("🚀 INICIANDO SISTEMA AVANÇADO DE SCORING AURUM")
    
    try:
        # 1. INICIALIZAR SISTEMA DE SCORING
        scoring_system = AurumScoringSystem()
        
        # 2. CARREGAR DADOS DO DATAFRAME MESTRE (Unificado)
        ratios_path = "data/aurum_master_features.parquet" # <- ESTA É A MUDANÇA
        
        if not Path(ratios_path).exists():
            logger.error(f"❌ Arquivo de ratios não encontrado: {ratios_path}")
            logger.info("💡 Execute primeiro o cálculo dos ratios financeiros")
            return None
        
        ratios_df = pd.read_parquet(ratios_path)
        logger.info(f"✅ Dados de ratios carregados: {ratios_df.shape}")
        
        # 3. CALCULAR SCORES INDIVIDUAIS
        individual_scores_df = scoring_system.calculate_individual_scores(ratios_df)
        
        # 4. CALCULAR SCORE COMPOSTO
        final_scores_df = scoring_system.calculate_composite_score(individual_scores_df)
        
        # 5. VALIDAR SISTEMA
        validation_report = scoring_system.validate_scoring_system(final_scores_df)
        
        # 6. SALVAR RESULTADOS
        output_dir = Path("data/aurum_final_scores")
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Salvar scores finais
        final_output_path = output_dir / "aurum_advanced_scores.parquet"
        final_scores_df.to_parquet(final_output_path, index=False)
        
        # Salvar apenas os mais recentes
        latest_scores = final_scores_df.sort_values(['CNPJ_CIA', 'DT_FIM_EXERC']).groupby('CNPJ_CIA').last().reset_index()
        latest_output_path = output_dir / "aurum_latest_advanced_scores.parquet"
        latest_scores.to_parquet(latest_output_path, index=False)
        
        # Salvar configuração
        config_path = scoring_system.save_scoring_configuration()
        
        # 7. RELATÓRIO FINAL
        print("\n" + "="*70)
        print("🎉 SISTEMA DE SCORING AURUM - RESULTADOS FINAIS")
        print("="*70)
        
        basic_stats = validation_report['basic_stats']
        distribution = validation_report['distribution']
        
        print(f"\n📊 ESTATÍSTICAS GERAIS:")
        print(f"   • Empresas no dataset: {basic_stats['total_companies']:,}")
        print(f"   • Empresas com score:  {basic_stats['companies_with_scores']:,}")
        print(f"   • Score médio: {basic_stats['score_mean']:.2f}")
        print(f"   • Score mediano: {basic_stats['score_median']:.2f}")
        print(f"   • Melhor score: {basic_stats['score_max']:.2f}")
        print(f"   • Pior score: {basic_stats['score_min']:.2f}")
        
        print(f"\n📈 DISTRIBUIÇÃO DAS NOTAS:")
        print(f"   • Nota A (Excelente): {distribution['grade_A']:3d} empresas")
        print(f"   • Nota B (Boa):       {distribution['grade_B']:3d} empresas") 
        print(f"   • Nota C (Regular):   {distribution['grade_C']:3d} empresas")
        print(f"   • Nota D (Baixa):     {distribution['grade_D']:3d} empresas")
        print(f"   • Nota E (Muito Baixa): {distribution['grade_E']:3d} empresas")
        
        print(f"\n🥇 TOP 10 EMPRESAS POR QUALIDADE:")
        top_10 = latest_scores.nlargest(10, 'aurum_quality_score')[
            ['DENOM_CIA', 'aurum_quality_score', 'quality_grade', 'quality_description']
        ]
        
        for i, (_, row) in enumerate(top_10.iterrows(), 1):
            print(f"   {i:2d}. {row['DENOM_CIA'][:35]:35} {row['aurum_quality_score']:6.2f} ({row['quality_grade']})")
        
        print(f"\n📋 CONTRIBUIÇÃO DAS MÉTRICAS:")
        metric_contrib = validation_report['metric_contribution']
        for metric_name, contrib_info in list(metric_contrib.items())[:6]:  # Mostrar top 6
            diff = contrib_info['actual_contribution'] - contrib_info['weight']
            diff_symbol = "+" if diff > 0 else ""
            print(f"   • {metric_name:15}: {contrib_info['actual_contribution']:5.1f}% (peso: {contrib_info['weight']:.1f}%) {diff_symbol}{diff:+.1f}%")
        
        weight_summary = validation_report['weight_summary']
        print(f"\n⚖️  RESUMO DE PESOS:")
        print(f"   • Peso teórico total: {weight_summary['total_theoretical_weight']}%")
        print(f"   • Peso real total:    {weight_summary['total_actual_weight']}%")
        
        print(f"\n💾 ARQUIVOS GERADOS:")
        print(f"   • Scores completos: {final_output_path}")
        print(f"   • Scores recentes:  {latest_output_path}")
        print(f"   • Configuração:     {config_path}")
        
        return {
            'scoring_system': scoring_system,
            'final_scores': final_scores_df,
            'latest_scores': latest_scores,
            'validation_report': validation_report
        }
        
    except Exception as e:
        logger.error(f"❌ Erro no sistema de scoring: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

# Função para análise rápida de métricas
def analyze_metric_importance():
    """Analisa a importância de cada métrica no sistema de scoring"""
    scoring_system = AurumScoringSystem()
    
    print("\n🔍 ANÁLISE DE IMPORTÂNCIA DAS MÉTRICAS")
    print("="*50)
    
    metrics_by_category = {
        'RENTABILIDADE': ['ROE', 'ROA', 'MARGEM_EBIT', 'MARGEM_LIQUIDA', 'MARGEM_BRUTA'],
        'SOLVÊNCIA': ['ALAVANCAGEM', 'DIVIDA_PL', 'LIQUIDEZ_CORRENTE', 'GIRO_ATIVO'],
        'CRESCIMENTO': ['CRESC_RECEITA', 'CRESC_LUCRO'],
        'EFICIÊNCIA': ['SENTIMENT_MEDIO', 'VOLATILIDADE']
    }
    
    total_weight = 0
    for category, metrics in metrics_by_category.items():
        category_weight = sum(
            scoring_system.scoring_metrics[metric].weight 
            for metric in metrics 
            if metric in scoring_system.scoring_metrics
        )
        total_weight += category_weight
        
        print(f"\n📊 {category}: {category_weight*100:.1f}%")
        for metric in metrics:
            if metric in scoring_system.scoring_metrics:
                metric_config = scoring_system.scoring_metrics[metric]
                print(f"   • {metric:20} {metric_config.weight*100:5.1f}% - {metric_config.description}")
    
    print(f"\n📈 SOMA TOTAL: {total_weight*100:.1f}%")

if __name__ == "__main__":
    # Executar sistema completo
    results = run_advanced_scoring_system()
    
    # Mostrar análise de importância
    analyze_metric_importance()
    
    print("\n🎯 PRÓXIMOS PASSOS SUGERIDOS:")
    print("   1. Analisar correlação entre scores e performance futura")
    print("   2. Ajustar pesos baseado em backtesting histórico") 
    print("   3. Implementar ajustes setoriais específicos")
    print("   4. Criar dashboard de monitoramento dos scores")

2025-12-11 00:28:10,561 - INFO - 🚀 INICIANDO SISTEMA AVANÇADO DE SCORING AURUM
2025-12-11 00:28:10,563 - INFO - ✅ Sistema de scoring inicializado com 13 métricas
2025-12-11 00:28:10,566 - INFO - 📊 Soma dos pesos: 1.000
2025-12-11 00:28:10,568 - ERROR - ❌ Arquivo de ratios não encontrado: data/aurum_master_features.parquet
2025-12-11 00:28:10,568 - INFO - 💡 Execute primeiro o cálculo dos ratios financeiros
2025-12-11 00:28:10,570 - INFO - ✅ Sistema de scoring inicializado com 13 métricas
2025-12-11 00:28:10,570 - INFO - 📊 Soma dos pesos: 1.000



🔍 ANÁLISE DE IMPORTÂNCIA DAS MÉTRICAS

📊 RENTABILIDADE: 47.0%
   • ROE                   18.0% - Return on Equity - Eficiência do capital próprio
   • ROA                   12.0% - Return on Assets - Eficiência dos ativos
   • MARGEM_EBIT            8.0% - Margem Operacional - Ebit/Receita
   • MARGEM_LIQUIDA         5.0% - Margem Líquida - Lucro/Receita
   • MARGEM_BRUTA           4.0% - Margem Bruta - Lucro Bruto/Receita

📊 SOLVÊNCIA: 33.0%
   • ALAVANCAGEM            9.0% - Alavancagem Total - Passivo/Ativo
   • DIVIDA_PL              9.0% - Dívida/Patrimônio Líquido
   • LIQUIDEZ_CORRENTE      8.0% - Liquidez Corrente - Ativo Circulante/Passivo Circulante
   • GIRO_ATIVO             7.0% - Giro do Ativo - Receita/Ativo Total

📊 CRESCIMENTO: 10.0%
   • CRESC_RECEITA          5.0% - Crescimento da Receita (anual)
   • CRESC_LUCRO            5.0% - Crescimento do Lucro Líquido (anual)

📊 EFICIÊNCIA: 10.0%
   • SENTIMENT_MEDIO        5.0% - Sentimento Médio de Notícias
   • VOLATILIDA